# Import materials from files in multiple formats (CIF, POSCAR, etc.)

This notebook uses ASE python package to extract structural information from files in multiple formats (CIF, POSCAR, etc., as supported by ASE). Some formats, like `espresso-in` and `espresso-out` can be inferred from the file content.

<h2 style="color:green">Usage</h2>

1. Upload files to `user_uploads` folder: open (double-click) the folder in the left sidebar, then click "Upload" and select the files to upload or just drag-and-drop files onto the sidebar.
1. Click "Run" > "Run All Cells" to run all cells
1. In case of format detection error, please correct the file format extension and try again

## Methodology

The following happens in the script below:

1. Install the required packages
1. The files are extracted from `user_uploads` folder assuming their extensions represent the format - e.g. `SiO2.poscar`.
1. Structural information is read from files into ASE Atoms objects.
1. ASE Atoms objects are converted to `poscar` format 
1. `poscar` structures are converted to ESSE
1. The results are passed to the outside runtime

## 1. Set Parameters

In [2]:
# Upload files to this folder
FOLDER_PATH = "./user_uploads"
# Attempt to guess the format from file extension
# If set to specific format, it will only accept that format
ENFORCED_FORMAT = None
# If set to true, the file extension will be included in the resulting material name
USE_FILE_NAME_NO_EXTENSION = False
# If set to true, the supported formats will be printed below
SHOW_SUPPORTED_FORMATS = False

## 2. Install Packages

In [ ]:
from jupyterlite.utils import install_packages

await install_packages("import_materials_from_files.ipynb")

## 3. Data Processing

### 3.1. Read data from files

In [ ]:
import os
from pathlib import Path
from ase.io import read

materials = []
unreadable_files = []
file_names = os.listdir(FOLDER_PATH)

for file_name in file_names:
    file_path = os.path.join(FOLDER_PATH, file_name)
    try:
        atoms = read(file_path, format=ENFORCED_FORMAT)

        atoms.info["file_name"] = Path(file_name).stem if USE_FILE_NAME_NO_EXTENSION else file_name
        materials.append(atoms)
        
    except Exception as e:
        print(e)
        unreadable_files.append(file_name)
        continue

### 3.2. Preview the data

In [ ]:
print(f"Successfully read {len(materials)} files")
print(f"Unreadable files: {unreadable_files}. ")


### 3.3. Troubleshoot data

In [ ]:
# Uncomment to see the list of supported formats and their file extensions
from ase.io.formats import ioformats
import pandas as pd

if len(unreadable_files) > 0 or SHOW_SUPPORTED_FORMATS:
    print(f"Unreadable files found: {unreadable_files}. See formats/extensions below.")
    data = [[frmt.name, frmt.extensions, frmt.description] for frmt in ioformats.values()]
    dataframe = pd.DataFrame(data, columns=["Format Name", "File Extensions", "Description"])
    print(dataframe.to_markdown())

### 3.4. Convert to ESSE format

In [ ]:
from src.utils import ase_to_poscar
from express import ExPrESS

def convert_ase_entry_to_esse(ase_entry):
    poscar = ase_to_poscar(ase_entry)
    kwargs = {
        "structure_string": poscar,
        "structure_format": "poscar"
    }

    handler = ExPrESS("structure", **kwargs)
    esse = handler.property("material", **kwargs)
    
    esse["name"] = ase_entry.info["file_name"]
    
    return esse

esse_entries = list(map(convert_ase_entry_to_esse, materials))

### 3.5. Preview the data

In [ ]:
from src.utils import poscar_to_ase
from ase.visualize import view
from ase.io import write
from ase.build import make_supercell
from IPython.display import Image

def visualize_material(material, index: int, number_of_repetitions: int = 3):
    """
    Visualize the material using ASE's visualization tool
    Repeat the unit cell to make it easier to see.

    Args:
        material: The material to visualize (Ase.Atoms object)
        index: The index of the material
        number_of_repetitions: The number of unit cell repetitions to visualize
    """
    # Set the number of unit cell repetition for the structure to make it easier to see
    n = number_of_repetitions
    material_repeat = make_supercell(material, [[n,0,0],[0,n,0],[0,0,n]])
    filename = f"material-{index}.png"
    write(filename, material_repeat)
    img = Image(filename=filename)
    print(filename, "-", material.symbols)
    display(img)

for idx, material in enumerate(materials):
    visualize_material(material, idx)

## 4. Pass data to the outside runtime

In [ ]:
from jupyterlite.utils import set_data

output_materials = esse_entries
set_data("materials", output_materials)

In [30]:
from ase.io.formats import ioformats
import pandas as pd

data = [[frmt.name, frmt.extensions, frmt.description] for frmt in ioformats.values()]
dataframe = pd.DataFrame(data, columns=["Format Name", "File Extensions", "Description"])
print(dataframe.to_markdown())


|    | Format Name        | File Extensions   | Description                            |
|---:|:-------------------|:------------------|:---------------------------------------|
|  0 | abinit-in          | []                | ABINIT input file                      |
|  1 | abinit-out         | []                | ABINIT output file                     |
|  2 | aims               | ['in']            | FHI-aims geometry file                 |
|  3 | aims-output        | []                | FHI-aims output                        |
|  4 | bundletrajectory   | []                | ASE bundle trajectory                  |
|  5 | castep-castep      | ['castep']        | CASTEP output file                     |
|  6 | castep-cell        | ['cell']          | CASTEP geom file                       |
|  7 | castep-geom        | ['geom']          | CASTEP trajectory file                 |
|  8 | castep-md          | ['md']            | CASTEP molecular dynamics file         |
|  9 | castep-phonon 